[View in Colaboratory](https://colab.research.google.com/github/heumchri/erfnet_pytorch/blob/master/testPretrainedERFNet.ipynb)

# general

In [0]:
#!kill -9 -1

## utilization monitoring

In [0]:
#requirements for gpu and ram usage

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


In [0]:
#gpu and ram usage

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
#running processes

#!ps -aux
!ps -aux | grep python

In [0]:
#storage usage

!df -h 

## increase shm for multithreading to work

In [0]:
cd /etc/

In [0]:
%%writefile fstab
tmpfs /dev/shm tmpfs defaults,size=2G 0 0

In [0]:
!mount -o remount /dev/shm

In [0]:
#storage usage

!df -h 

## mount google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
cd /content/

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
ls

# Install requirements

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [0]:
!pip3 install numpy matplotlib torchvision Pillow visdom

# clone repo

In [0]:
!git clone https://github.com/heumchri/erfnet_pytorch.git

In [0]:
cd /content/erfnet_pytorch/

In [0]:
!git pull

# test forward time
20 classes:

decoder batch 1: 0.100

decoder batch 16: 0.069

encoder batch 1: 0.069

encoder batch 16: 0.044

binary: 

decoder batch 1: 

decoder batch 16: 

encoder batch 1: 

encoder batch 16: 

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
!python3 eval_forwardTime.py --batch-size 1

In [0]:
!python3 eval_forwardTime.py --batch-size 16 --onlyEncoder

# test pretrained model (val dataset, nn upsampling) 0.976

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
!python eval_cityscapes_server.py --datadir /content/datasets/cityscapes/ --subset val

## visualize results

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/erfnet_pytorch/eval/save_results/val/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test pretrained model (val dataset, bilinear upsampling) 0.976

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
import numpy as np
import torch
import os
import importlib

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from dataset import cityscapes
from erfnet import ERFNet
from transform import Relabel, ToLabel, Colorize


NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512),
    ToTensor(),
    #Normalize([.485, .456, .406], [.229, .224, .225]),
])
target_transform_cityscapes = Compose([
    Resize(512),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

cityscapes_trainIds2labelIds = Compose([
    Relabel(19, 255),  
    Relabel(18, 33),
    Relabel(17, 32),
    Relabel(16, 31),
    Relabel(15, 28),
    Relabel(14, 27),
    Relabel(13, 26),
    Relabel(12, 25),
    Relabel(11, 24),
    Relabel(10, 23),
    Relabel(9, 22),
    Relabel(8, 21),
    Relabel(7, 20),
    Relabel(6, 19),
    Relabel(5, 17),
    Relabel(4, 13),
    Relabel(3, 12),
    Relabel(2, 11),
    Relabel(1, 8),
    Relabel(0, 7),
    Relabel(255, 0),
    ToPILImage(),
    #Resize(1024, Image.NEAREST),
])
def evalDecoderValset():
    up = torch.nn.Upsample(scale_factor=2, mode='bilinear')
    up = up.cuda()

    modelpath = "../trained_models/" + "erfnet.py"
    weightspath = "../trained_models/" + "erfnet_pretrained.pth"

    print ("Loading model: " + modelpath)
    print ("Loading weights: " + weightspath)

    #Import ERFNet model from the folder
    #Net = importlib.import_module(modelpath.replace("/", "."), "ERFNet")
    model = ERFNet(NUM_CLASSES)

    model = torch.nn.DataParallel(model)
    if (not False):
        model = model.cuda()

    #model.load_state_dict(torch.load(args.state))
    #model.load_state_dict(torch.load(weightspath)) #not working if missing key

    def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                 continue
            own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath))
    print ("Model and weights LOADED successfully")

    model.eval()

    if(not os.path.exists("/content/datasets/cityscapes/")):
        print ("Error: datadir could not be loaded")


    loader = DataLoader(cityscapes("/content/datasets/cityscapes/", input_transform_cityscapes, target_transform_cityscapes, subset="val"),
        num_workers=4, batch_size=1, shuffle=False)

    for step, (images, labels, filename, filenameGt) in enumerate(loader):
        
        if (not False):
            images = images.cuda()
            #labels = labels.cuda()

        inputs = Variable(images, volatile=True)
        #targets = Variable(labels, volatile=True)
        outputs = model(inputs,only_encode=False)
        outputs = up(outputs)

        label = outputs[0].max(0)[1].byte().cpu().data
        label_cityscapes = cityscapes_trainIds2labelIds(label.unsqueeze(0))
        #print (numpy.unique(label.numpy()))  #debug

        filenameSave = "./save_results/" + filename[0].split("leftImg8bit/")[1]
        os.makedirs(os.path.dirname(filenameSave), exist_ok=True)
        #image_transform(label.byte()).save(filenameSave)
        label_cityscapes.save(filenameSave)

        print (step, filenameSave)


In [0]:
evalDecoderValset()

## visualize results

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/erfnet_pytorch/eval/save_results/val/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test pretrained model (val dataset, nn upsampling, only encoder) 0.070

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
!python eval_cityscapes_server.py --datadir /content/datasets/cityscapes/ --subset val --onlyEncoder

## visualize results

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/erfnet_pytorch/eval/save_results/val/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test pretrained model (val dataset, bilinear upsampling, only encoder) 0.049

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
import numpy as np
import torch
import os
import importlib

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from dataset import cityscapes
from erfnet import ERFNet
from transform import Relabel, ToLabel, Colorize


NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512),
    ToTensor(),
    #Normalize([.485, .456, .406], [.229, .224, .225]),
])
target_transform_cityscapes = Compose([
    Resize(512),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

cityscapes_trainIds2labelIds = Compose([
    Relabel(19, 255),  
    Relabel(18, 33),
    Relabel(17, 32),
    Relabel(16, 31),
    Relabel(15, 28),
    Relabel(14, 27),
    Relabel(13, 26),
    Relabel(12, 25),
    Relabel(11, 24),
    Relabel(10, 23),
    Relabel(9, 22),
    Relabel(8, 21),
    Relabel(7, 20),
    Relabel(6, 19),
    Relabel(5, 17),
    Relabel(4, 13),
    Relabel(3, 12),
    Relabel(2, 11),
    Relabel(1, 8),
    Relabel(0, 7),
    Relabel(255, 0),
    ToPILImage(),
    #Resize(1024, Image.NEAREST),
])
def evalEncoderValset():
    up = torch.nn.Upsample(scale_factor=16, mode='bilinear')
    up = up.cuda()

    modelpath = "../trained_models/" + "erfnet.py"
    weightspath = "../trained_models/" + "erfnet_pretrained.pth"

    print ("Loading model: " + modelpath)
    print ("Loading weights: " + weightspath)

    #Import ERFNet model from the folder
    #Net = importlib.import_module(modelpath.replace("/", "."), "ERFNet")
    model = ERFNet(NUM_CLASSES)

    model = torch.nn.DataParallel(model)
    if (not False):
        model = model.cuda()

    #model.load_state_dict(torch.load(args.state))
    #model.load_state_dict(torch.load(weightspath)) #not working if missing key

    def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                 continue
            own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath))
    print ("Model and weights LOADED successfully")

    model.eval()

    if(not os.path.exists("/content/datasets/cityscapes/")):
        print ("Error: datadir could not be loaded")


    loader = DataLoader(cityscapes("/content/datasets/cityscapes/", input_transform_cityscapes, target_transform_cityscapes, subset="val"),
        num_workers=4, batch_size=1, shuffle=False)

    for step, (images, labels, filename, filenameGt) in enumerate(loader):
        
        if (not False):
            images = images.cuda()
            #labels = labels.cuda()

        inputs = Variable(images, volatile=True)
        #targets = Variable(labels, volatile=True)
        outputs = model(inputs,only_encode=True)
        outputs = up(outputs)

        label = outputs[0].max(0)[1].byte().cpu().data
        label_cityscapes = cityscapes_trainIds2labelIds(label.unsqueeze(0))
        #print (numpy.unique(label.numpy()))  #debug

        filenameSave = "./save_results/" + filename[0].split("leftImg8bit/")[1]
        os.makedirs(os.path.dirname(filenameSave), exist_ok=True)
        #image_transform(label.byte()).save(filenameSave)
        label_cityscapes.save(filenameSave)

        print (step, filenameSave)


In [0]:
evalEncoderValset()

## visualize results

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/erfnet_pytorch/eval/save_results/val/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test retrained model (val dataset, bilinear upsampling, only encoder) 0.491
**epoch 2: 0.491**

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
import numpy as np
import torch
import os
import importlib

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from dataset import cityscapes
from erfnet import ERFNet
from transform import Relabel, ToLabel, Colorize


NUM_CHANNELS = 3
NUM_CLASSES = 2

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512),
    ToTensor(),
    #Normalize([.485, .456, .406], [.229, .224, .225]),
])
target_transform_cityscapes = Compose([
    Resize(512),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

cityscapes_trainIds2labelIds = Compose([
    Relabel(0, 7),
    Relabel(1, 0),
    ToPILImage(),
    #Resize(1024, Image.NEAREST),
])
def evalEncoderValset():
    up = torch.nn.Upsample(scale_factor=16, mode='bilinear')
    up = up.cuda()

    modelpath = "../trained_models/" + "erfnet.py"
    weightspath = "/content/drive/esp_new/" + "model_encoder_best.pth"

    print ("Loading model: " + modelpath)
    print ("Loading weights: " + weightspath)

    #Import ERFNet model from the folder
    #Net = importlib.import_module(modelpath.replace("/", "."), "ERFNet")
    model = ERFNet(NUM_CLASSES)

    model = torch.nn.DataParallel(model)
    if (not False):
        model = model.cuda()

    #model.load_state_dict(torch.load(args.state))
    #model.load_state_dict(torch.load(weightspath)) #not working if missing key

    def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                 continue
            own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath))
    print ("Model and weights LOADED successfully")

    model.eval()

    if(not os.path.exists("/content/datasets/cityscapes/")):
        print ("Error: datadir could not be loaded")


    loader = DataLoader(cityscapes("/content/datasets/cityscapes/", input_transform_cityscapes, target_transform_cityscapes, subset="val"),
        num_workers=4, batch_size=1, shuffle=False)

    for step, (images, labels, filename, filenameGt) in enumerate(loader):
        
        if (not False):
            images = images.cuda()
            #labels = labels.cuda()

        inputs = Variable(images, volatile=True)
        #targets = Variable(labels, volatile=True)
        outputs = model(inputs,only_encode=True)
        outputs = up(outputs)

        label = outputs[0].max(0)[1].byte().cpu().data
        label_cityscapes = cityscapes_trainIds2labelIds(label.unsqueeze(0))
        #print (numpy.unique(label.numpy()))  #debug

        filenameSave = "./save_results/" + filename[0].split("leftImg8bit/")[1]
        os.makedirs(os.path.dirname(filenameSave), exist_ok=True)
        #image_transform(label.byte()).save(filenameSave)
        label_cityscapes.save(filenameSave)

        print (step, filenameSave)


In [0]:
evalEncoderValset()

## visualize results

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/erfnet_pytorch/eval/save_results/val/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# retrain model (binary)

##retrain encoder

### start training

In [0]:
cd /content/erfnet_pytorch/train/

In [0]:
!python3 main_binary.py --savedir /content/drive/esp_new/encoder/ --datadir /content/datasets/cityscapes/ --num-epochs 150 --batch-size 6 

###resume training

In [0]:
cd /content/erfnet_pytorch/train

In [0]:
!python3 main_binary.py --savedir /content/drive/esp_new/encoder/ --datadir /content/datasets/cityscapes/ --num-epochs 150 --batch-size 6 --resume --iouTrain

##retrain decoder

### start training

###resume training

# Run evaluations

In [0]:
cd /content/erfnet_pytorch/eval/

In [0]:
!python eval_cityscapes_color.py --datadir /content/datasets/cityscapes/ --subset val --num-workers 0

In [0]:
from IPython.display import Image
Image('./save_color/val/munster/munster_000000_000019_leftImg8bit.png')

In [0]:
!python eval_iou.py --datadir /content/datasets/cityscapes/ --subset val --num-workers 0

In [0]:
!python eval_cityscapes_server.py --datadir /content/datasets/cityscapes/ --subset val

In [0]:
from IPython.display import Image
Image('./save_results/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png')